**Мультиагентная система для технического интервью**

## Архитектура:
- **Observer Agent**: Анализирует ответы, проверяет факты, дает инструкции
- **Interviewer Agent**: Ведет диалог с кандидатом
- **Manager Agent**: Принимает финальное решение о найме

In [1]:
!pip install -q langchain langchain-google-genai google-generativeai

Будем использовать модель от Google - gemini-3-flash-preview

In [2]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [3]:
import json
import os
from datetime import datetime
from typing import List, Dict, Optional
from dataclasses import dataclass, asdict
from enum import Enum

from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage


@dataclass
class CandidateProfile:
    name: str
    position: str
    target_grade: str
    experience: str


@dataclass
class ConversationTurn:
    turn_id: int
    agent_visible_message: str
    user_message: str
    internal_thoughts: str


## Промпты для агентов

In [4]:
OBSERVER_SYSTEM_PROMPT = """Ты - Observer Agent в системе технического интервью.

ТВОЯ РОЛЬ:
- Анализируй ответы кандидата на технические вопросы
- Проверяй факты и выявляй галлюцинации/ложную информацию
- Оценивай уровень знаний и уверенность
- Давай инструкции Interviewer Agent, куда вести беседу дальше

ВАЖНО:
- Ты СКРЫТ от кандидата - твои мысли не видны пользователю
- Если кандидат говорит неправду или бред - ОБЯЗАТЕЛЬНО укажи это
- Если кандидат уходит off-topic - укажи Interviewer вернуть к интервью
- Отслеживай контекст - не позволяй задавать повторные вопросы

ФОРМАТ ОТВЕТА:
Анализ: [твоя оценка ответа кандидата]
Факт-чек: [правда/ложь/галлюцинация]
Инструкция Interviewer: [что делать дальше - усложнить/упростить/уточнить/вернуть к теме]
Рекомендуемая тема: [о чем спросить следующим]
"""

INTERVIEWER_SYSTEM_PROMPT = """Ты - Interviewer Agent, который ведет техническое интервью.

ТВОЯ РОЛЬ:
- Задавай технические вопросы кандидату
- Адаптируй сложность вопросов на основе ответов
- Веди дружелюбный, но профессиональный диалог
- Следуй инструкциям Observer Agent

ПРАВИЛА:
- НЕ задавай вопросы, на которые кандидат уже ответил
- Если кандидат задает встречный вопрос - ОТВЕТЬ на него кратко, потом продолжи интервью
- Если кандидат уходит off-topic - ВЕЖЛИВО верни к теме
- Если кандидат говорит бред - НЕ СОГЛАШАЙСЯ, мягко поправь
- Адаптируй сложность: отличные ответы → сложнее, слабые ответы → проще/подсказка

СТИЛЬ:
- Дружелюбный, но деловой
- Короткие вопросы (1-3 предложения)
- Без excessive вежливости

Текущая позиция: {position}
Целевой грейд: {target_grade}
Опыт кандидата: {experience}
"""

MANAGER_SYSTEM_PROMPT = """Ты - Manager Agent, принимающий финальное решение о найме.

ТВОЯ РОЛЬ:
- Анализируй весь диалог интервью
- Оцени технические навыки (hard skills)
- Оцени коммуникативные навыки (soft skills)
- Вынеси вердикт: Grade + Hiring Decision + Confidence
- Составь персональный Roadmap для развития

КРИТЕРИИ ОЦЕНКИ:
Junior: Базовые знания, может решать простые задачи под контролем
Middle: Уверенные знания, самостоятельность, может наставлять джунов
Senior: Глубокая экспертиза, архитектурное мышление, лидерство

Confidence Score:
- 80-100%: Очень уверен в оценке
- 60-79%: Средняя уверенность
- <60%: Низкая уверенность (мало данных)

ФОРМАТ: Строго структурированный JSON
"""

## Мультиагентная система с рефлексией

In [5]:
class InterviewCoachSystem:

    def __init__(self, api_key: str, model: str = "gemini-3-flash-preview"):
        self.observer_llm = ChatGoogleGenerativeAI(
            model=model,
            google_api_key=api_key,
            temperature=0.3,
            convert_system_message_to_human=True
        )

        self.interviewer_llm = ChatGoogleGenerativeAI(
            model=model,
            google_api_key=api_key,
            temperature=0.7,
            convert_system_message_to_human=True
        )

        self.manager_llm = ChatGoogleGenerativeAI(
            model=model,
            google_api_key=api_key,
            temperature=0.2,
            convert_system_message_to_human=True
        )

        # История диалога
        self.conversation_history: List[ConversationTurn] = []
        self.candidate_profile: Optional[CandidateProfile] = None
        self.turn_counter = 0

    def initialize_interview(self, name: str, position: str,
                            target_grade: str, experience: str) -> str:
        self.candidate_profile = CandidateProfile(
            name=name,
            position=position,
            target_grade=target_grade,
            experience=experience
        )

        greeting_prompt = f"""
Поприветствуй кандидата {name}, который пришел на позицию {position} ({target_grade}).
Опыт: {experience}

Задачи:
1. Кратко поздороваться
2. Попросить рассказать о себе
3. Дать понять, что это техническое интервью

Будь дружелюбным, но кратким (2-3 предложения).
"""

        response = self.interviewer_llm.invoke([
            SystemMessage(content=INTERVIEWER_SYSTEM_PROMPT.format(
                position=position,
                target_grade=target_grade,
                experience=experience
            )),
            HumanMessage(content=greeting_prompt)
        ])

        greeting = response.content

        self.conversation_history.append(ConversationTurn(
            turn_id=self.turn_counter,
            agent_visible_message=greeting,
            user_message="",
            internal_thoughts=f"[System]: Интервью инициализировано для {name}"
        ))
        self.turn_counter += 1

        return greeting

    def process_user_response(self, user_message: str) -> Dict[str, str]:
        """Обработка ответа через мультиагентную систему"""

        # Проверка на команду завершения
        if "стоп" in user_message.lower() and ("интервью" in user_message.lower() or
                                                 "игра" in user_message.lower()):
            return self._finalize_interview()

        # ШАГ 1: Observer анализирует ответ
        observer_analysis = self._observer_analyze(user_message)

        # ШАГ 2: Interviewer генерирует ответ на основе анализа
        interviewer_response = self._interviewer_respond(user_message, observer_analysis)

        # Логирование
        internal_thoughts = (
            f"[Observer]: {observer_analysis}\n"
            f"[Interviewer Decision]: Генерирую ответ на основе анализа Observer"
        )

        self.conversation_history.append(ConversationTurn(
            turn_id=self.turn_counter,
            agent_visible_message=interviewer_response,
            user_message=user_message,
            internal_thoughts=internal_thoughts
        ))
        self.turn_counter += 1

        return {
            'visible_response': interviewer_response,
            'internal_thoughts': internal_thoughts
        }

    def _observer_analyze(self, user_message: str) -> str:
        """Observer Agent анализирует ответ"""
        recent_context = self._get_recent_context(3)

        analysis_prompt = f"""
Проанализируй последний ответ кандидата.

КОНТЕКСТ:
{recent_context}

НОВЫЙ ОТВЕТ:
{user_message}

Позиция: {self.candidate_profile.position}
Целевой грейд: {self.candidate_profile.target_grade}
"""

        response = self.observer_llm.invoke([
            SystemMessage(content=OBSERVER_SYSTEM_PROMPT),
            HumanMessage(content=analysis_prompt)
        ])


    def _interviewer_respond(self, user_message: str, observer_analysis: str) -> str:
        """Interviewer Agent генерирует ответ"""
        recent_context = self._get_recent_context(5)

        response_prompt = f"""
КОНТЕКСТ:
{recent_context}

ОТВЕТ КАНДИДАТА:
{user_message}

АНАЛИЗ OBSERVER:
{observer_analysis}

ЗАДАЧА: Сгенерируй следующий ход на основе анализа.
"""

        response = self.interviewer_llm.invoke([
            SystemMessage(content=INTERVIEWER_SYSTEM_PROMPT.format(
                position=self.candidate_profile.position,
                target_grade=self.candidate_profile.target_grade,
                experience=self.candidate_profile.experience
            )),
            HumanMessage(content=response_prompt)
        ])

        return response.content

    def _get_recent_context(self, n: int = 3) -> str:
        """Получить последние N ходов"""
        recent_turns = self.conversation_history[-n:]

        context_parts = []
        for turn in recent_turns:
            context_parts.append(
                f"Ход {turn.turn_id}:\n"
                f"Интервьюер: {turn.agent_visible_message}\n"
                f"Кандидат: {turn.user_message}\n"
            )

        return "\n".join(context_parts)

    def _finalize_interview(self) -> Dict[str, str]:




        """Финализация - генерация фидбэка через Manager"""
        full_transcript = self._get_recent_context(len(self.conversation_history))

        feedback_prompt = f"""
Проанализируй полное интервью и составь фидбэк.

ВАЖНО:
Интервью МОГЛО завершиться без технических ответов кандидата
(например, кандидат сразу завершил интервью или не дал ни одного ответа).

ЕСЛИ:
- нет ни одного осмысленного ответа кандидата
- или интервью завершено сразу после приветствия

ТО:
- НЕ выдумывай знания, навыки или поведение
- укажи, что данных недостаточно
- используй нейтральные и честные формулировки

ПРАВИЛА:
- Никаких галлюцинаций
- Оценка должна строго соответствовать данным
- Если данных недостаточно — явно укажи это

ВЕРНИ СТРОГО JSON (без markdown, без ```json):
{{
  "grade": "Junior/Middle/Senior",
  "hiring_recommendation": "No Hire/Hire/Strong Hire",
  "confidence_score": 0-100,
  "technical_review": {{
    "confirmed_skills": ["навык1"],
    "knowledge_gaps": [
      {{"topic": "тема", "correct_answer": "ответ"}}
    ]
  }},
  "soft_skills": {{
    "clarity": "оценка",
    "honesty": "оценка",
    "engagement": "оценка"
  }},
  "roadmap": ["тема1"],
  "documentation_links": ["https://..."]
}}
"""

        response = self.manager_llm.invoke([
            SystemMessage(content=MANAGER_SYSTEM_PROMPT),
            HumanMessage(content=feedback_prompt)
        ])

        feedback_json = response.content

        # Очистка от markdown
        if "```json" in feedback_json:
            feedback_json = feedback_json.split("```json")[1].split("```")[0].strip()
        elif "```" in feedback_json:
            feedback_json = feedback_json.split("```")[1].split("```")[0].strip()

        self.final_feedback = feedback_json

        return {
            'visible_response': "Спасибо! Генерирую фидбэк...",
            'internal_thoughts': f"[Manager]: {feedback_json}",
            'final_feedback': feedback_json
        }

    def save_log(self, filename: str = "interview_log.json"):

        log_data = {
            "participant_name": self.candidate_profile.name,
            "turns": [asdict(turn) for turn in self.conversation_history],
            "final_feedback": self.final_feedback
        }

        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(log_data, f, ensure_ascii=False, indent=2)

        print(f"Лог сохранен: {filename}")
        return filename

## Удобная обертка для использования

In [6]:
class InterviewSession:

    def __init__(self, api_key: str):
        self.system = InterviewCoachSystem(api_key)
        self.is_active = False
        self.final_feedback: Optional[str] = None

    def start(self, name: str, position: str, target_grade: str, experience: str):
        print("=" * 60)
        print("MULTI-AGENT INTERVIEW COACH")
        print("=" * 60)

        greeting = self.system.initialize_interview(
            name, position, target_grade, experience
        )

        print(f"\nИнтервьюер: {greeting}\n")
        self.is_active = True

    def chat(self, show_internal: bool = True):
        """Интерактивный диалог через input()"""
        if not self.is_active:
            print("Сначала вызовите start()")
            return

        print("Начните диалог. Для завершения напишите: 'стоп интервью'\n")

        while self.is_active:
            try:
                user_input = input("Вы: ").strip()

                if not user_input:
                    continue

                result = self.system.process_user_response(user_input)

                if show_internal:
                    print("\nВНУТРЕННИЕ МЫСЛИ АГЕНТОВ:")
                    print("-" * 60)
                    print(result["internal_thoughts"])
                    print("-" * 60)

                print(f"\nИнтервьюер: {result['visible_response']}\n")

                if "final_feedback" in result:
                    print("\nФИНАЛЬНЫЙ ФИДБЭК")
                    print("=" * 60)
                    print(result["final_feedback"])
                    self.is_active = False

            except KeyboardInterrupt:
                print("\nДиалог прерван пользователем")
                self.is_active = False
    def save_log(self, filename: str = "interview_log.json"):
        """Сохранить лог интервью"""
        return self.system.save_log(filename)


## ЗАПУСК ИНТЕРВЬЮ

In [7]:
session = InterviewSession(GOOGLE_API_KEY)

session.start(
    name="Яхонтов Максим Витальевич",
    position="Machine Learning Engineer",
    target_grade="Senior ",
    experience=""
)

session.chat(show_internal=True)

MULTI-AGENT INTERVIEW COACH

Интервьюер: [{'type': 'text', 'text': 'Здравствуйте, Максим Витальевич! Рад познакомиться, сегодня мы проведем техническое интервью на позицию Senior Machine Learning Engineer. \n\nДля начала расскажите, пожалуйста, о себе и своем наиболее значимом опыте в области ML.', 'extras': {'signature': 'EvIKCu8KAXLI2nynb3ktOvoBQBKp7ivLWNBhgWrwHtkhbdt5vxmb6ZCgGCuF5+PII8DKk6SOLvk4WFs0oOEhm7fQE0Bx3RI5uBo5JkrI9WE/J1W483OT5ZQ56qpVj3K3TKLs8uSBNA/DEEM/sKp0hanbDBBn1lJ4LtZDxS+4E3BnTRhdacp2VYQ9/PWb86dSWlHz9NqJ2qO9G/LpI9bATx56vAfqtig5JhFqiTA4eJNqLyaPWKdtesu6pfjWZROeoWAG0NSVci2o+OpP4YvqicE27KnIx0srO8hWaWAab3UTur7Wmo3o6GOprRpd7Z8zsI+a8tuNSyESgCmY5ekrbmysR6Kl8MIlvA6d4msyZxHMxcTxGHLxLCiM0IlMEtvmdhr7oElfwjP9mN1aJkwhbd0D0QS6XJ+FMPydFN4UbyJ5eHduKVsQqV/WI338tylZXu/EY5X4RrO+vq0DkfVx6kjNAdhBe37gpcRAorKmldAR7ySMkLQkYhKDK0NrSd3W9MwfRn9YuIJk8N/lM/LHWIzwumQh6DW5/Dck8NqC+R+1CKM9tYr+LjNvZ7jFID+IQth/Mpw3YaU0b1S+NX2nnfReajA530qhZzPZlBeDtntcKbEvnojDxkXUn8/FEtjtE0l8H8o4EjLTW39AlcN0NmkRT7f3gdBe+gi

## Сохранение лога

In [8]:
log_file = session.save_log("interview_alex_junior.json")

from google.colab import files
files.download(log_file)

Лог сохранен: interview_alex_junior.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Просмотр лога

In [9]:
with open(log_file, 'r', encoding='utf-8') as f:
    log_data = json.load(f)

print(json.dumps(log_data, ensure_ascii=False, indent=2))

{
  "participant_name": "Яхонтов Максим Витальевич",
  "turns": [
    {
      "turn_id": 0,
      "agent_visible_message": [
        {
          "type": "text",
          "text": "Здравствуйте, Максим Витальевич! Рад познакомиться, сегодня мы проведем техническое интервью на позицию Senior Machine Learning Engineer. \n\nДля начала расскажите, пожалуйста, о себе и своем наиболее значимом опыте в области ML.",
          "extras": {
            "signature": "EvIKCu8KAXLI2nynb3ktOvoBQBKp7ivLWNBhgWrwHtkhbdt5vxmb6ZCgGCuF5+PII8DKk6SOLvk4WFs0oOEhm7fQE0Bx3RI5uBo5JkrI9WE/J1W483OT5ZQ56qpVj3K3TKLs8uSBNA/DEEM/sKp0hanbDBBn1lJ4LtZDxS+4E3BnTRhdacp2VYQ9/PWb86dSWlHz9NqJ2qO9G/LpI9bATx56vAfqtig5JhFqiTA4eJNqLyaPWKdtesu6pfjWZROeoWAG0NSVci2o+OpP4YvqicE27KnIx0srO8hWaWAab3UTur7Wmo3o6GOprRpd7Z8zsI+a8tuNSyESgCmY5ekrbmysR6Kl8MIlvA6d4msyZxHMxcTxGHLxLCiM0IlMEtvmdhr7oElfwjP9mN1aJkwhbd0D0QS6XJ+FMPydFN4UbyJ5eHduKVsQqV/WI338tylZXu/EY5X4RrO+vq0DkfVx6kjNAdhBe37gpcRAorKmldAR7ySMkLQkYhKDK0NrSd3W9MwfRn9YuIJk8N/lM/LHWIzwumQh